# 1、字符串输出解析器 StrOutputParser

In [3]:
from langchain_core.output_parsers import StrOutputParser
# 1、获取大模型
from langchain_openai import ChatOpenAI
import os
import dotenv

# 加载环境变量
dotenv.load_dotenv()

os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

chat_model = ChatOpenAI(
    model="gpt-4o-mini"
)

# 2、调用大模型
response = chat_model.invoke("什么是大语言模型？")

print(type(response))  # AIMessage
# print(response)

# 3、如何获取一个字符串类型的结果呢？
# 方式①：自己调用输出结果的content
# print(response.content)

# 方式②：使用 StrOutputParser
parser = StrOutputParser()
str_response = parser.invoke(response)
print(type(str_response))
print(str_response)

<class 'langchain_core.messages.ai.AIMessage'>
<class 'str'>
大语言模型（Large Language Model，LLM）是指一种通过处理大量文本数据来学习语言的高级人工智能模型。这些模型基于深度学习技术，特别是变换器（Transformer）架构，旨在理解和生成自然语言。大语言模型具有以下几个主要特点：

1. **规模庞大**：这些模型通常由数亿到数千亿个参数组成，能够捕捉语言中的复杂模式和结构。

2. **预训练和微调**：大语言模型通常首先在大型文本语料库上进行预训练，以学习语言的基本知识；然后可以通过微调适应特定的任务，如情感分析、问答、翻译等。

3. **上下文理解**：由于使用了大量的数据，语言模型能够在给定上下文的基础上生成连贯且相关的文本。

4. **应用广泛**：大语言模型可以用于各种自然语言处理（NLP）任务，包括但不限于文本生成、摘要、对话系统、信息检索等。

5. **可迁移性**：经过预训练的模型可以很容易地适应特定领域的任务，减少了从头开始训练的时间和资源消耗。

总之，大语言模型是一种强大的工具，能够在多个场景中处理和生成自然语言，为各种应用提供支持。


# 2、JSON解析器 JsonOutputParser


方式1：用户借助自己的提示词指明返回JSON格式

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
# 1、获取大模型
from langchain_openai import ChatOpenAI
import os
import dotenv

# 加载环境变量
dotenv.load_dotenv()

os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

chat_model = ChatOpenAI(
    model="gpt-4o-mini"
)

# 根据提示词模板创建提示词
chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个靠谱的{role}"),
        ("human", "{question}")
    ]
)

# 2、调用大模型
response = chat_model.invoke(
    chat_prompt_template.invoke(
        {
            "role": "人工智能专家",
            "question": "人工智能用英语怎么说？问题用q表示, 答案用a表示, 返回一个JSON格式的数据"
        }
    )
)

print(response.content)

# 获取一个JsonOutputParser的实例
parser = JsonOutputParser()

json_result = parser.invoke(response)
print(json_result)

<class 'langchain_core.messages.ai.AIMessage'>
```json
{
  "q": "人工智能用英语怎么说？",
  "a": "Artificial Intelligence"
}
```
{'q': '人工智能用英语怎么说？', 'a': 'Artificial Intelligence'}


方式2：借助JsonOutputParser的get_format_instructions(), 生成格式说明, 指导模型输出JSON结构

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
import os
import dotenv

# 加载环境变量
dotenv.load_dotenv()

os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# 1、获取大模型
chat_model = ChatOpenAI(
    model="gpt-4o-mini"
)

joke_query = "告诉我一个笑话。"

# 定义Json解析器
parser = JsonOutputParser()

# 2、定义提示词模版
# 注意，提示词模板中需要部分格式化解析器的格式要求format_instructions
prompt_template = PromptTemplate.from_template(
    template="回答用户的查询.\n满足的格式为{format_instructions}\n问题为{question}\n",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt = prompt_template.invoke({"question": joke_query})

# 3、调用大模型获取返回结果
response = chat_model.invoke(prompt)
json_result = parser.invoke(response)

print(json_result)

{'joke': '为什么计算机很冷？因为它们总是打开窗口！'}


## 知识的拓展：管道符


针对于举例1改造

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
# 1、获取大模型
from langchain_openai import ChatOpenAI
import os
import dotenv

# 加载环境变量
dotenv.load_dotenv()

os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

chat_model = ChatOpenAI(
    model="gpt-4o-mini"
)

# 根据提示词模板创建提示词
chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个靠谱的{role}"),
        ("human", "{question}")
    ]
)

# 获取一个JsonOutputParser的实例
parser = JsonOutputParser()

# 2、调用大模型
## 写法1：
# response = chat_model.invoke(
#     chat_prompt_template.invoke(
#         {
#             "role": "人工智能专家",
#             "question": "人工智能用英语怎么说？问题用q表示, 答案用a表示, 返回一个JSON格式的数据"
#         }
#     )
# )
# 
# json_result = parser.invoke(response)
# print(json_result)

## 写法2：
chain = chat_prompt_template | chat_model | parser
json_result_chain = chain.invoke(
    {
        "role": "人工智能专家",
        "question": "人工智能用英语怎么说？问题用q表示, 答案用a表示, 返回一个JSON格式的数据"
    }
)
print(json_result_chain)

{'q': '人工智能用英语怎么说？', 'a': 'Artificial Intelligence'}


针对于举例2改造：

In [11]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
import os
import dotenv

# 加载环境变量
dotenv.load_dotenv()

os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# 1、获取大模型
chat_model = ChatOpenAI(
    model="gpt-4o-mini"
)

joke_query = "告诉我一个笑话。"

# 定义Json解析器
parser = JsonOutputParser()

# 2、定义提示词模版
# 注意，提示词模板中需要部分格式化解析器的格式要求format_instructions
prompt_template = PromptTemplate.from_template(
    template="回答用户的查询.\n满足的格式为{format_instructions}\n问题为{question}\n",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

## 写法1：
# prompt = prompt_template.invoke({"question": joke_query})
# # 3、调用大模型获取返回结果
# response = chat_model.invoke(prompt)
# json_result = parser.invoke(response)

## 写法2：
chain = prompt_template | chat_model | parser
json_result_chain = chain.invoke(
    {"question": joke_query}
)

print(json_result_chain)

{'joke': 'Why did the scarecrow win an award? Because he was outstanding in his field!'}
